In [1]:
import math
import nltk
import scipy
import string
import re
import numpy as np
import pandas as pd
import seaborn as sns

from tqdm import tqdm
from scipy.stats import randint
from wordcloud import WordCloud
from multiprocessing import Pool
from nltk.corpus import stopwords
from scipy.stats import loguniform

from sklearn.decomposition import PCA
from nltk.stem.porter import PorterStemmer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RandomizedSearchCV,RepeatedStratifiedKFold,GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score,roc_auc_score, roc_curve, precision_score, recall_score
# from scikitplot.metrics import plot_roc_curve as auc_roc
import matplotlib.pyplot as plt
from IPython.display import display
plt.rcParams['figure.figsize'] = [20,6]
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
import warnings 
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv('Tweets.csv', header=0)

In [3]:
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [4]:
df.describe()

,textID,text,selected_text,sentiment
count,27481,27480,27480,27481
unique,27481,27480,22463,3
top,d9639a0a97,"Masson I`ll give you the link, it`s quite handy",good,neutral
freq,1,1,199,11118


In [5]:
df.drop(['selected_text', 'textID'], axis=1, inplace=True)

In [6]:
df.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [7]:
df.shape

(27481, 2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       27480 non-null  object
 1   sentiment  27481 non-null  object
dtypes: object(2)
memory usage: 429.5+ KB


In [9]:
df.dropna(inplace=True)
second_df = df.copy(deep=True)
second_df

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...
27476,wish we could come see u on Denver husband l...,negative
27477,I`ve wondered about rake to. The client has ...,negative
27478,Yay good for both of you. Enjoy the break - y...,positive
27479,But it was worth it ****.,positive


In [10]:
df[df.duplicated()]

,text,sentiment


In [11]:
counter = 0
r,c = second_df.shape

df_nodup = df.drop_duplicates()
df_nodup.reset_index(drop=True, inplace=True)

In [12]:
df_nodup.head()

,text,sentiment
0,"I`d have responded, if I were going",neutral
1,Sooo SAD I will miss you here in San Diego!!!,negative
2,my boss is bullying me...,negative
3,what interview! leave me alone,negative
4,"Sons of ****, why couldn`t they put them on t...",negative


In [13]:
df_filter = df_nodup.copy()

def preprocessor(text):
    #text = re.sub('[http:,https:]','',text)
    text = re.sub('[^a-zA-Z]',' ',text)
    text = text.lower()
    text = text.strip()
    text = ''.join([i for i in text if i in string.ascii_lowercase+' '])
    text = ' '.join([word for word in text.split() if word.isalnum()])
    text = ' '.join([WordNetLemmatizer().lemmatize(word,pos='v') for word in text.split()])    
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])
    return text

In [ ]:
df_filter['text'] = df_nodup['text'].apply(preprocessor)

df_filter.head()

In [ ]:
porter=PorterStemmer()
target = 'sentiment'
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
df_vector=TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None,tokenizer=tokenizer_porter,use_idf=True,norm='l2',smooth_idf=True)
label=df_filter[target].values
features=df_vector.fit_transform(df_filter.text)

In [ ]:
print(label)
print(features)

In [ ]:
print('\033[1mSentiment Distibution'.center(55))

plt.pie(df_filter[target].value_counts(), labels=['Neutral','Positive','Negative'], counterclock=False, shadow=True, 
        explode=[0,0,0.1], autopct='%1.1f%%', radius=1.5, startangle=0)
plt.show()

In [ ]:
df_rl = df_filter.copy()
df_rl['review_length'] = 0


df_rl['review_length'] = df_rl['text'].apply(lambda x: len(x))

plt.figure(figsize=[20,8])
sns.boxplot(x=label,y='review_length', data=df_rl, hue=label,palette=['blue','red','green'])
plt.title('Text Length')
plt.xlabel("Sentiment")
plt.ylabel("Length of reviews")
plt.show()

In [ ]:
neut_df = second_df[second_df[target]=='neutral']['text']
pos_df = second_df[second_df[target]=='positive']['text']

neg_df = second_df[second_df[target]== 'negative']['text']

In [ ]:
def freq_text(text_df):     # returns all occurences of words in text_df
 
    combi_text=""
    for x in text_df.values:
        combi_text+=' '.join(x.split())
    combi_text  = [x for x in combi_text.split() if len(x) >3 and x not in stopwords.words('english')]  
    return combi_text

In [ ]:
def freq_df(text_count_df,senti):    # returns the count of each word in text_count_df obtained from previous function
    fredis = nltk.FreqDist(neut_text)
    fredis_df = pd.DataFrame({senti: list(fredis.keys()),
                      'Count': list(fredis.values())})
    fredis_df = fredis_df.sort_values(by='Count',ascending=False)
    return fredis_df

In [ ]:
def plot_freq_dis(text_count_df,senti):  # plots freq_df
    sns.barplot(data=text_count_df[:10],x=senti,y='Count')

In [ ]:
neut_df = neut_df.apply(preprocessor)
neut_df

In [ ]:
sen = 'Neutral'
print('\033[1mFrequency of Words owing to Neutral texts'.center(150))

neut_text = freq_text(neut_df)
neut_freq_df = freq_df(neut_text,sen)
plot_freq_dis(neut_frq_df,sen)

In [ ]:
pos_df = pos_df.apply(preprocessor)

In [ ]:
sen ='Positive'
print('\033[1mFrequency of Words owing to Positive texts'.center(150))
pos_df_text = freq_text(pos_df)
pos_frq_df = freq_df(pos_df_text,sen)
plot_freq_dis(pos_frq_df,sen)

In [ ]:
neg_df = neg_df.apply(preprocessor)

In [ ]:
sen ='Negative'
print('\033[1mFrequency of Words owing to Negative texts'.center(150))
neut_text = freq_text(neg_df)
neut_frq_df = freq_df(neut_text,sen)
plot_freq_dis(neut_frq_df,sen)